In [1]:
import urllib.request
from bs4 import BeautifulSoup as bs
from configparser import ConfigParser
import psycopg2
import pandas as pd

This cell defines the psycopg connection

In [2]:
connection = None
try:
    parser = ConfigParser()
    parser.read("database.ini")
    db = {}
    if parser.has_section("postgresql"):
        params = parser.items("postgresql")
        for param in params:
            db[param[0]] = param[1]
    connection = psycopg2.connect(**db)
    crsr = connection.cursor()
    crsr.execute('SELECT version()')
    crsr.close()
except(Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if connection is not None:
        connection.close()

This cell is for all the links for the lists

In [11]:
urls = []
# urls.append("https://digitaldreamdoor.com/pages/best_rb-ballads-x.html")
# urls.append("https://digitaldreamdoor.com/pages/best_balladsddd.html")
# urls.append("https://digitaldreamdoor.com/pages/best_metal-songs-x.html")
# urls.append("https://digitaldreamdoor.com/pages/best_guitarriff-x.html")
# urls.append("https://digitaldreamdoor.com/pages/best_songs70s-x.html")
urls.append("https://digitaldreamdoor.com/pages/best_disco-songs-x.html")
urls.append("https://digitaldreamdoor.com/pages/best_undersongs-x.html")
urls.append("https://digitaldreamdoor.com/pages/best_songs-folk-rock-x.html")


In [12]:
for url in urls:
    page = urllib.request.urlopen(url)
    soup = bs(page)
    
    connection = psycopg2.connect(**db)
    crsr = connection.cursor() 
    

    list_name = soup.title.get_text()
    list_count = list_name.split(' ')[0]
    list_id = None
    crsr.execute(f"SELECT * FROM LIST WHERE NAME LIKE '{list_name}'")
    if len(crsr.fetchall()) == 0:
        print("list doesn't exist")
        crsr.execute(f"INSERT INTO LIST(NAME, COUNT) VALUES('{list_name}', '{list_count}')")
        connection.commit()
    crsr.execute(f"SELECT * FROM LIST WHERE NAME LIKE '{list_name}'")
    list_id = crsr.fetchone()[0]


    data = []
    divs = soup.findAll('div', class_='list')
    
    for i, div in enumerate(divs):
      songs = div.get_text(separator='').split('\n')
      cleaned_songs = [song.strip() for song in songs if song.strip()]
    
      for song in cleaned_songs:
        number_title_artist = song.split(' - ')
        if len(number_title_artist) == 2:
          title, artist = number_title_artist
          data.append({'Title': title, 'Artist': artist})
    df = pd.DataFrame(data)
    

    for index, row in df.iterrows():
        artist = row["Artist"]
        query = "SELECT artistid FROM ARTIST WHERE name LIKE %s"
        crsr.execute(query, (artist,))
        artist_id = crsr.fetchone()  
        if artist_id is None:
            insert_query = "INSERT INTO ARTIST (name) VALUES (%s)"
            crsr.execute(insert_query, (artist,))
            crsr.execute(query, (artist,))  
            artist_id = crsr.fetchone()[0]
        else:
            artist_id = artist_id[0]
            
            
        title = row["Title"].split('. ')
        song_rank = title[0]
        song_name = title[1]
        song_query = "SELECT SongID FROM SONG WHERE NAME LIKE %s"
        crsr.execute(song_query, (song_name,))
        song_id = crsr.fetchone()
        
        if song_id is None:
            query_song_insert = "INSERT INTO SONG(ARTISTID, NAME) VALUES (%s, %s)"
            crsr.execute(query_song_insert, (artist_id, song_name))
            crsr.execute(song_query, (song_name,))
            song_id = crsr.fetchone()[0]
        else:
            song_id = song_id[0]
        
        crsr.execute(f"SELECT Song_ListID FROM Song_List WHERE SongID = %s AND ListID = %s", (song_id, list_id))
        song_list_id = crsr.fetchone()
        if song_list_id is None:
            crsr.execute(f"INSERT INTO Song_List(SongID, ListID, Ranking) VALUES (%s, %s, %s)", (song_id, list_id, song_rank))
        
        connection.commit()
        

list doesn't exist
list doesn't exist
list doesn't exist


IndexError: list index out of range

In [5]:
    crsr.close()
    connection.close()